In [1]:
# setup of the notebook
%load_ext Cython
%pylab inline
from iminuit import Minuit, describe

Populating the interactive namespace from numpy and matplotlib


In [2]:
%%cython --force
cimport cython
import numpy as np
cimport numpy as np

@cython.embedsignature(True)  # put function signature in pydoc so `describe` can extract it
def cython_f(double x,double y,double z):
    return (x - 1.) ** 2 + (y - 2.) ** 2 + (z - 3.) ** 2 + 1.

In [3]:
m = Minuit(cython_f, pedantic=False, errordef=1)
fmin, param = m.migrad()

In [4]:
class LeastSquares:
    def __init__(self, model, x, y):
        self.model = model  # model predicts y for given x
        self.x = array(x)
        self.y = array(y)

    def __call__(self, *par):  # par are a variable number of model parameters
        ym = self.model(self.x, *par)
        chi2 = sum((self.y - ym)**2)
        return chi2

In [5]:
def line(x, a, b):  # simple straight line model with explicit parameters
    return a + b * x

lsq = LeastSquares(line,
                   [1, 2, 3, 4, 5],
                   [2, 4, 6, 8, 10])

try:
    describe(lsq)  # this raises a TypeError, because signature cannot be read
except TypeError as e:
    print(e)

Unable to obtain function signature


In [6]:
from iminuit.util import make_func_code
# get the args from line and strip 'x'
par_names = describe(line)[1:]
lsq.func_code = make_func_code(par_names)

# now we get the right answer
describe(lsq)

['a', 'b']

In [7]:
m = Minuit(lsq, pedantic=False)
fmin, param = m.migrad()

In [8]:
class LeastSquares:  # override the class with a better one
    def __init__(self, model, x, y):
        self.model = model  # model predicts y for given x
        self.x = array(x)
        self.y = array(y)
        self.func_code = make_func_code(describe(self.model)[1:])

    def __call__(self, *par):  # par are a variable number of model parameters
        ym = self.model(self.x, *par)
        chi2 = sum((self.y - ym)**2)
        return chi2

In [9]:
lsq = LeastSquares(line,
                   [1, 2, 3, 4, 5],
                   [2, 4, 6, 8, 10])
describe(lsq)  # works!

['a', 'b']

In [10]:
%%cython
# example with absolutely no signature, generated with cython
# note: you can add a signature with the decorate @cython.embedsignature(True) as shown above
def nosig_f(x,y):
    return x**2+(y-4)**2

In [11]:
try:
    describe(nosig_f)  # raises error
except TypeError as e:
    print(e)

Unable to obtain function signature


In [12]:
# you can always force parameters, like so
m = Minuit(nosig_f, forced_parameters=('x','y'), pedantic=False)

In [14]:
m.migrad()

------------------------------------------------------------------
| FCN = 0                       |      Ncalls=14 (38 total)      |
| EDM = 0 (Goal: 1E-05)         |            up = 1.0            |
------------------------------------------------------------------
|  Valid Min.   | Valid Param.  | Above EDM | Reached call limit |
------------------------------------------------------------------
|     True      |     True      |   False   |       False        |
------------------------------------------------------------------
| Hesse failed  |   Has cov.    | Accurate  | Pos. def. | Forced |
------------------------------------------------------------------
|     False     |     True      |   True    |   True    | False  |
------------------------------------------------------------------
------------------------------------------------------------------------------------------
|   | Name |   Value   | Hesse Err | Minos Err- | Minos Err+ | Limit-  | Limit+  | Fixed |
------------------------------------------------------------------------------------------
| 0 | x    |    0.0    |    1.0    |            |            |         |         |       |
| 1 | y    |    4.0    |    1.0    |            |            |         |         |       |
------------------------------------------------------------------------------------------